### Real World application 2 (Chapitre 3)

In [5]:
# 1) Create an index, explain your choice of key.

import pymongo

# Se connecter à la base de données MongoDB
client = pymongo.MongoClient('localhost', 27017)
mydb = client["pubmed_authors"]
collection = mydb["authors"]

# Créer un index sur la clé AND_ID
collection.create_index("AND_ID")

'AND_ID_1'

In [8]:
# 2) What is the average length of "pmid_list"

# Requête d'agrégation pour calculer la longueur moyenne de la liste "pmid_list"
pipeline = [
    {"$match": {"pmid_list": {"$type": "array"}}},  # Filtrer les documents avec pmid_list sous forme de liste
    {"$match": {"pmid_list": {"$ne": None}}},  # Filtrer les documents où pmid_list n'est pas null
    {"$unwind": "$pmid_list"},  # Dérouler la liste "pmid_list"
    {"$group": {"_id": None, "avg_pmid_count": {"$avg": 1}}}  # Calculer la moyenne du nombre de pmid_list
]

# Exécuter la requête d'agrégation
result = list(collection.aggregate(pipeline))

# Afficher le résultat
if result:
    average_length = result[0]["avg_pmid_count"]
    print("Average length of pmid_list:", average_length)
else:
    print("No documents found in the collection.")

Average length of pmid_list: 1.0


In [9]:
# 3) How many distinct affiliations are there ?

pipeline = [
    {"$match": {"oa04_affiliations": {"$exists": True, "$ne": []}}},  # Filter out documents with empty or non-existent oa04_affiliations field
    {"$unwind": "$oa04_affiliations"},  # Unwind the oa04_affiliations array
    {"$group": {"_id": "$oa04_affiliations.Affiliation"}},  # Group by the Affiliation field and count distinct values
    {"$group": {"_id": None, "count": {"$sum": 1}}}  # Count the distinct affiliations
]

# Execute the aggregation pipeline
result = list(collection.aggregate(pipeline))

# Extract the count from the result
if result:
    distinct_affiliations_count = result[0]["count"]
    print("Number of distinct affiliations:", distinct_affiliations_count)
else:
    print("No documents found with oa04_affiliations.")

Number of distinct affiliations: 318595


In [11]:
# 4) Find authors with atleast one "COM" AffiliationType

pipeline = [
    {"$unwind": "$oa04_affiliations"},  # Unwind the oa04_affiliations array
    {"$group": {"_id": "$AND_ID", "affiliation_types": {"$addToSet": "$oa04_affiliations.AffiliationType"}}},  # Group by author and collect distinct AffiliationTypes
    {"$match": {"affiliation_types.1": {"$exists": True}}}  # Match authors who have more than one distinct AffiliationType
]

# Execute the aggregation pipeline
result = list(collection.aggregate(pipeline))

# Count the authors who switched AffiliationType
count = len(result)

print("Number of authors who switched AffiliationType:", count)

Number of authors who switched AffiliationType: 21670


In [9]:
# 5) How many authors switched the AffiliationType ?

pipeline = [
    {"$match": {"oa04_affiliations": {"$elemMatch": {"AffiliationType": "COM"}}}},  # Filter documents with at least one "COM" AffiliationType
    {"$project": {"more_info": 1}},  # Project only the more_info field
    {"$unwind": "$more_info"},  # Unwind the more_info array
    {"$group": {"_id": "$more_info", "count": {"$sum": 1}}}  # Group by more_info and count occurrences
]


# Execute the aggregation pipeline and limit the result to the first 10 authors
result = list(collection.aggregate(pipeline))[:10]

# Extract the authors with at least one "COM" AffiliationType
if result:
    for author in result:
        print(author["_id"])
else:
    print("No authors found with at least one 'COM' AffiliationType.")


#Pour tout afficher :
# Execute the aggregation pipeline
#result = list(collection.aggregate(pipeline))

# Extract the authors with at least one "COM" AffiliationType
#if result:
#    for author in result:
#        print(author["_id"])
#else:
#    print("No authors found with at least one 'COM' AffiliationType.")

{'id': 693, 'PMID': 275, 'AND_ID': 2519679, 'AuOrder': 3, 'LastName': 'Schaefer', 'ForeName': 'U W', 'Initials': 'UW', 'Suffix': nan, 'AuNum': 4, 'PubYear': 1975, 'BeginYear': 1971}
{'id': 891, 'PMID': 361, 'AND_ID': 1648874, 'AuOrder': 1, 'LastName': 'Mehta', 'ForeName': 'R J', 'Initials': 'RJ', 'Suffix': nan, 'AuNum': 1, 'PubYear': 1975, 'BeginYear': 1968}
{'id': 1039, 'PMID': 413, 'AND_ID': 1683950, 'AuOrder': 1, 'LastName': 'Mürer', 'ForeName': 'E H', 'Initials': 'EH', 'Suffix': nan, 'AuNum': 3, 'PubYear': 1975, 'BeginYear': 1967}
{'id': 1146, 'PMID': 458, 'AND_ID': 2495621, 'AuOrder': 2, 'LastName': 'Chu', 'ForeName': 'S H', 'Initials': 'SH', 'Suffix': nan, 'AuNum': 3, 'PubYear': 1975, 'BeginYear': 1972}
{'id': 1414, 'PMID': 562, 'AND_ID': 7407821, 'AuOrder': 3, 'LastName': 'Goodman', 'ForeName': 'S A', 'Initials': 'SA', 'Suffix': nan, 'AuNum': 6, 'PubYear': 1975, 'BeginYear': 1972}
{'id': 1429, 'PMID': 570, 'AND_ID': 8580663, 'AuOrder': 2, 'LastName': 'Friedhoff', 'ForeName': 'A 

In [11]:
# 6) Find affiliation with the word "China" 

import re

# Define the regex pattern to match affiliations containing the word "China"
pattern = re.compile("China", re.IGNORECASE)

pipeline = [
    {"$unwind": "$oa04_affiliations"},  # Unwind the oa04_affiliations array
    {"$match": {"oa04_affiliations.Affiliation": {"$regex": pattern}}},  # Match affiliations containing the word "China"
    {"$project": {"_id": 0, "Affiliation": "$oa04_affiliations.Affiliation"}}  # Project only the Affiliation field
]

# Execute the aggregation pipeline
cursor = collection.aggregate(pipeline)

# Display the affiliations containing the word "China"
count = 0
for affiliation in cursor:
    print(affiliation["Affiliation"])
    count += 1

print("Total affiliations containing the word 'China':", count)

School of Medicine, Huzhou University, Huzhou Centeral Hospital, 759 Erhuan RD, Huzhou, Zhejing Province, 313000, China. Electronic address: aring2000@163.com.
Department of Basic Medical Science, Xi'an Medical University, China. suxingli@126.com
Department of Urology, Peking University First Hospital, Beijing, 100034, China. xuesongli123@sina.cn.
Department of Radiology, Jiangxi Province Medical Imaging Research Institute, the First Affiliated Hospital of Nanchang University, No.17, Yongwai Zheng Street, Donghu District, Nanchang, 330006, Jiangxi Province, People's Republic of China.
College of Life Sciences, Nanjing Normal University, Nanjing, China.
Department of Obstetrics and Gynecology, Peking University People's Hospital, No. 11, Xi-Zhi-Men South Street, Xi Cheng District, Beijing, 100044, China.
The Applied Plant Genomics Laboratory, National Key Laboratory of Crop Genetics and Germplasm Enhancement, Nanjing Agricultural University, Nanjing 210095, Jiangsu, China.
Key Laborator

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [17]:
# 7) Get the pmids of papers published in 2019

pipeline = [
    {"$unwind": "$more_info"},  # Unwind the more_info array
    {"$match": {"more_info.PubYear": 2019}},  # Match documents with a PubYear of 2019
    {"$project": {"_id": 0, "PMID": "$more_info.PMID"}}  # Project only the PMID field
]

# Execute the aggregation pipeline and limit the result to the first 10 PMIDs
cursor = collection.aggregate(pipeline, allowDiskUse=True)

# Collect the PMIDs, limited to the first 10
pmids = [doc["PMID"] for doc in cursor][:10]

# Display the PMIDs
print("PMIDs of papers published in 2019:")
for pmid in pmids:
    print(pmid)


#Pour tout afficher :
# Execute the aggregation pipeline
#cursor = collection.aggregate(pipeline)

# Collect the PMIDs
#pmids = [doc["PMID"] for doc in cursor]

# Display the PMIDs
#print("PMIDs of papers published in 2019:")
#for pmid in pmids:
#    print(pmid)

PMIDs of papers published in 2019:
30972486
30859659
31388046
31787945
31055204
30653778
30654984
30610279
31646943
30741784


In [15]:
# 8) Count the number of doc with "oa06_researcher_education" OR "oa04_affiliations" key and with the "oa06_researcher_education" AND "oa04_affiliations" .

pipeline = [
    {
        "$match": {
            "$or": [
                {"oa06_researcher_education": {"$exists": True}},
                {"oa04_affiliations": {"$exists": True}}
            ],
            "$and": [
                {"oa06_researcher_education": {"$exists": True}},
                {"oa04_affiliations": {"$exists": True}}
            ]
        }
    },
    {"$count": "total_documents"}
]

# Execute the aggregation pipeline
result = list(collection.aggregate(pipeline))

# Extract the count of documents
total_documents = result[0]["total_documents"] if result else 0

# Display the count
print("Number of documents with oa06_researcher_education or oa04_affiliations key and with both oa06_researcher_education and oa04_affiliations keys:", total_documents)

Number of documents with oa06_researcher_education or oa04_affiliations key and with both oa06_researcher_education and oa04_affiliations keys: 903


In [25]:
# 9) What's the average "BeginYear" of "oa06_researcher_education".

pipeline = [
    {"$unwind": "$oa06_researcher_education"},  # Dérouler le tableau oa06_researcher_education
    {"$project": {"BeginYear": "$oa06_researcher_education.BeginYear"}},  # Projeter les valeurs de BeginYear
]

result = list(collection.aggregate(pipeline))[:10]  # Limiter les résultats aux 10 premiers

for doc in result:
    print(doc)

#Pour tout afficher :
#pipeline = [
#    {"$unwind": "$oa06_researcher_education"},  # Unwind the oa06_researcher_education array
#    {"$project": {"BeginYear": "$oa06_researcher_education.BeginYear"}},  # Project the BeginYear values
#]

#result = list(collection.aggregate(pipeline))

#for doc in result:
#    print(doc)

{'_id': ObjectId('6242c7c5022d5c5d474a0453'), 'BeginYear': '2016'}
{'_id': ObjectId('6242c7c5022d5c5d474a048e'), 'BeginYear': '2007'}
{'_id': ObjectId('6242c7c5022d5c5d474a04a4'), 'BeginYear': '2011'}
{'_id': ObjectId('6242c7c5022d5c5d474a04bd'), 'BeginYear': '2016'}
{'_id': ObjectId('6242c7c5022d5c5d474a04bd'), 'BeginYear': '2007'}
{'_id': ObjectId('6242c7c5022d5c5d474a04bd'), 'BeginYear': '2012'}
{'_id': ObjectId('6242c7c5022d5c5d474a04d5'), 'BeginYear': '2013'}
{'_id': ObjectId('6242c7c5022d5c5d474a04df'), 'BeginYear': '2013'}
{'_id': ObjectId('6242c7c5022d5c5d474a04f0'), 'BeginYear': '2002'}
{'_id': ObjectId('6242c7c5022d5c5d474a0526'), 'BeginYear': '1999'}


In [23]:
# 10) Count the distinct country of "oa06_researcher_education"

pipeline = [
    {"$unwind": "$oa06_researcher_education"},  # Unwind the oa06_researcher_education array
    {"$group": {"_id": "$oa06_researcher_education.Country"}},  # Group by country
    {"$match": {"_id": {"$ne": None}}},  # Filter out null values
    {"$group": {"_id": None, "count": {"$sum": 1}}}  # Count the distinct countries
]

result = list(collection.aggregate(pipeline))

if result:
    distinct_countries_count = result[0]['count']
    print("Distinct countries count:", distinct_countries_count)
else:
    print("No data found.")

Distinct countries count: 101


In [29]:
# 11) Does the length of pmid_list and more_info always match ?


# Récupérer les 10 premiers documents de la collection
documents = collection.find().limit(10)

# Parcourir les documents et vérifier la correspondance des longueurs
for doc in documents:
    pmid_list_length = len(doc["pmid_list"])
    more_info_length = len(doc["more_info"])
    if pmid_list_length != more_info_length:
        print(f"Pour le document avec AND_ID {doc['AND_ID']}, la longueur de pmid_list ({pmid_list_length}) ne correspond pas à la longueur de more_info ({more_info_length})")
    else:
        print(f"Pour le document avec AND_ID {doc['AND_ID']}, la longueur de pmid_list ({pmid_list_length}) correspond à la longueur de more_info ({more_info_length})")


#Pour tout afficher :
# Récupérer tous les documents de la collection
#documents = collection.find()

# Parcourir les documents et vérifier la correspondance des longueurs
#for doc in documents:
#    pmid_list_length = len(doc["pmid_list"])
#    more_info_length = len(doc["more_info"])
#    if pmid_list_length != more_info_length:
#        print(f"Pour le document avec AND_ID {doc['AND_ID']}, la longueur de pmid_list ({pmid_list_length}) ne correspond pas à la longueur de more_info ({more_info_length})")
#    else:
#        print(f"Pour le document avec AND_ID {doc['AND_ID']}, la longueur de pmid_list ({pmid_list_length}) correspond à la longueur de more_info ({more_info_length})")

Pour le document avec AND_ID 16569575, la longueur de pmid_list (1) correspond à la longueur de more_info (1)
Pour le document avec AND_ID 1323299, la longueur de pmid_list (1) correspond à la longueur de more_info (1)
Pour le document avec AND_ID 8005653, la longueur de pmid_list (1) correspond à la longueur de more_info (1)
Pour le document avec AND_ID 697768, la longueur de pmid_list (1) correspond à la longueur de more_info (1)
Pour le document avec AND_ID 5325855, la longueur de pmid_list (3) correspond à la longueur de more_info (3)
Pour le document avec AND_ID 17358905, la longueur de pmid_list (1) correspond à la longueur de more_info (1)
Pour le document avec AND_ID 18124378, la longueur de pmid_list (1) correspond à la longueur de more_info (1)
Pour le document avec AND_ID 4280730, la longueur de pmid_list (2) correspond à la longueur de more_info (2)
Pour le document avec AND_ID 9102178, la longueur de pmid_list (1) correspond à la longueur de more_info (1)
Pour le document 

In [33]:
# 12) Does the length of pmid_list and "oa04_affiliations" always match ?

# Récupérer les 10 premiers documents de la collection
documents = collection.find().limit(10)

# Parcourir les documents et vérifier la correspondance des longueurs
for doc in documents:
    pmid_list_length = len(doc.get("pmid_list", []))
    oa04_affiliations_length = len(doc.get("oa04_affiliations", []))
    
    # Check if the lengths match
    if pmid_list_length == oa04_affiliations_length:
        print(f"For document with _id: {doc['_id']}, lengths match.")
    else:
        print(f"For document with _id: {doc['_id']}, lengths do not match.")

#Pour tout afficher : 
# Iterate through each document in the collection
#for doc in collection.find():
#    pmid_list_length = len(doc.get("pmid_list", []))
#    oa04_affiliations_length = len(doc.get("oa04_affiliations", []))
    
    # Check if the lengths match
#    if pmid_list_length == oa04_affiliations_length:
#        print(f"For document with _id: {doc['_id']}, lengths match.")
#    else:
#        print(f"For document with _id: {doc['_id']}, lengths do not match.")

For document with _id: 6242c7c5022d5c5d474a044e, lengths do not match.
For document with _id: 6242c7c5022d5c5d474a044f, lengths match.
For document with _id: 6242c7c5022d5c5d474a0450, lengths match.
For document with _id: 6242c7c5022d5c5d474a0451, lengths match.
For document with _id: 6242c7c5022d5c5d474a0452, lengths do not match.
For document with _id: 6242c7c5022d5c5d474a0453, lengths do not match.
For document with _id: 6242c7c5022d5c5d474a0454, lengths do not match.
For document with _id: 6242c7c5022d5c5d474a0455, lengths do not match.
For document with _id: 6242c7c5022d5c5d474a0456, lengths match.
For document with _id: 6242c7c5022d5c5d474a0457, lengths match.


# 13) Sandbox exercise: think of a problematic and try to answer it.

Le problème ici est d'identifier les auteurs dont l'affiliation indiquée dans les informations supplémentaires (more_info) ne correspond pas à une affiliation répertoriée dans les affiliations associées à l'article (oa04_affiliations). Pour résoudre ce problème, nous devons comparer les affiliations des auteurs avec les affiliations de l'article et signaler toute incohérence.

Voici comment nous pouvons aborder ce problème :

1/Parcourir chaque document.
2/Extraire les listes d'affiliations des 3/auteurs (more_info) et celles de l'article (oa04_affiliations).
4/Pour chaque auteur, comparer son 5/affiliation avec chaque affiliation de l'article.
6/Si une correspondance est trouvée, passer à l'auteur suivant.
7/Si aucune correspondance n'est trouvée pour un auteur, signaler l'incohérence.

In [ ]:
# 13) Sandbox exercise: think of a problematic and try to answer it.

import pymongo

# Se connecter à la base de données MongoDB
client = pymongo.MongoClient('localhost', 27017)
mydb = client["pubmed_authors"]
collection = mydb["authors"]

# Créer un index sur la clé AND_ID
collection.create_index("AND_ID")

# Parcourir chaque document
for document in collection.find():
    pmid_list = document.get('pmid_list', [])
    more_info = document.get('more_info', [])
    oa04_affiliations = document.get('oa04_affiliations', [])

    # Vérifier si les longueurs de pmid_list et more_info correspondent
    matching_lengths = len(pmid_list) == len(more_info)
    if not matching_lengths:
        print(f"Les longueurs de pmid_list et more_info pour AND_ID {document['AND_ID']} ne correspondent pas.")

    # Vérifier si les longueurs de pmid_list et oa04_affiliations correspondent
    matching_lengths_affiliations = len(pmid_list) == len(oa04_affiliations)
    if not matching_lengths_affiliations:
        print(f"Les longueurs de pmid_list et oa04_affiliations pour AND_ID {document['AND_ID']} ne correspondent pas.")

    # Vérifier les affiliations des auteurs
    for info in more_info:
        author_affiliation = info.get('Affiliation', '')
        # Vérifier si l'affiliation de l'auteur est dans les affiliations de l'article
        match_found = False
        for affiliation_info in oa04_affiliations:
            if author_affiliation == affiliation_info.get('Affiliation', ''):
                match_found = True
                break
        # Si aucune correspondance n'est trouvée, signaler l'incohérence
        if not match_found:
            print(f"Incohérence potentielle pour l'auteur {info['LastName']} {info['ForeName']}: "
                  f"L'affiliation '{author_affiliation}' n'a pas été trouvée dans les affiliations de l'article.")